Flip image to get more training data (Aeffectilfe [..] without keypoints)


# Head pose estimation

Names: Javier Covas Llompart, Lukas Koberg

The goal is to develop a neural notwork, that is able to detect the pose of a head (the 3 angles). Therfor pytorch is used.

## Setup

In [62]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchsummary import summary
from torchvision.models import resnet50, resnet152
from torchvision.transforms.functional import pil_to_tensor
from PIL import Image
import torchvision.transforms as T
import os
import re
#from google.colab import drive
#drive.mount('/content/gdrive/', force_remount=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [2]:
basepath = "Advanced Topics Computer Vision/Short Project/"

In [3]:
basepath = "Data/"

In [4]:
def compute_euler_angles_from_rotation_matrices(rotation_matrices, full_range=False):
    batch = rotation_matrices.shape[0]
    R = rotation_matrices
    sy = torch.sqrt(R[:,0,0]*R[:,0,0]+R[:,1,0]*R[:,1,0])
    singular = sy<1e-6
    singular = singular.float()

    '''2023.01.15'''
    for i in range(len(sy)):  # expand y (yaw angle) range into (-180, 180)
        if R[i, 0, 0] < 0 and full_range:
            sy[i] = -sy[i]

    x = torch.atan2(R[:,2,1], R[:,2,2])
    y = torch.atan2(-R[:,2,0], sy)
    z = torch.atan2(R[:,1,0],R[:,0,0])

    xs = torch.atan2(-R[:,1,2], R[:,1,1])
    ys = torch.atan2(-R[:,2,0], sy)
    zs = R[:,1,0]*0

    gpu = rotation_matrices.get_device()
    if gpu < 0:
        out_euler = torch.autograd.Variable(torch.zeros(batch,3)).to(torch.device('cpu'))
    else:
        out_euler = torch.autograd.Variable(torch.zeros(batch,3)).to(torch.device('cuda:%d' % gpu))
    out_euler[:,0] = x*(1-singular)+xs*singular
    out_euler[:,1] = y*(1-singular)+ys*singular
    out_euler[:,2] = z*(1-singular)+zs*singular
    # print('out_euler', out_euler)

    return out_euler

## Load Datasets and Preperation

In [54]:
class BIWI_Dataset(Dataset):
    def __init__(self): #Example code
        # convert into PyTorch tensors and remember them
        
        self.X = self.read_data()[0]
        self.y = self.read_data()[1]

    def __len__(self):
        # this should return the size of the dataset
        return len(self.X)

    def __getitem__(self, idx):
        # this should return one sample from the dataset
        #features = pil_to_tensor(Image.open(self.X[idx])).float()
        img_arr = np.array(Image.open(self.X[idx]))
        data = img_arr.astype(np.float64)
        data = 255 * data
        #img_arr = data.astype(np.uint8)
        features = self.transform(img_arr)
        target = self.y[idx, 0]
        return features, target
    
    transform = T.Compose([
        T.ToPILImage(),
        T.Resize(([64, 49])),
        T.ToTensor()])
    
    def read_data(self):
        list_angle_data = []
        list_picture_data = []

        #folder_nr = "01"
        for i in range(1, 2):
            if i <10:
                folder_nr = "0" + str(i)
            else:
                folder_nr = str(i)
            for file in os.listdir(basepath + "BIWI/" + folder_nr):
                if file.endswith(".png"):
                    list_picture_data.append(basepath + "BIWI/" + folder_nr + "/" + file)
                    f = open(basepath + "BIWI/" + folder_nr + "/frame_" + file.split("_")[1] + "_pose.txt", "r")
                    file_data = f.read()
                    file_data_split = file_data.split()
                    matrix = np.array(file_data_split[0:9], dtype=np.float32).reshape(3, 3)
                    list_angle_data.append(matrix)
        #print((np.stack(list_angle_data, axis=0).shape))
        angles = compute_euler_angles_from_rotation_matrices(torch.tensor(np.stack(list_angle_data, axis=0), dtype=torch.float32))
        #print(angles*180/np.pi)
        return list_picture_data, angles


In [55]:
dataset = BIWI_Dataset()
print(dataset.__getitem__(1))

(tensor([[[0.7333, 0.7961, 0.8196,  ..., 0.5373, 0.5255, 0.4902],
         [0.8824, 0.9529, 0.9725,  ..., 0.5922, 0.5804, 0.5412],
         [0.8118, 0.8549, 0.8431,  ..., 0.5922, 0.5843, 0.5451],
         ...,
         [0.3137, 0.3176, 0.3020,  ..., 0.4863, 0.2549, 0.2431],
         [0.3373, 0.3529, 0.2824,  ..., 0.4745, 0.2431, 0.2392],
         [0.3098, 0.3294, 0.2235,  ..., 0.4275, 0.2118, 0.2196]],

        [[0.7255, 0.7843, 0.8118,  ..., 0.5098, 0.5020, 0.4627],
         [0.8745, 0.9451, 0.9686,  ..., 0.5686, 0.5569, 0.5176],
         [0.8235, 0.8588, 0.8510,  ..., 0.5725, 0.5608, 0.5216],
         ...,
         [0.2157, 0.2196, 0.2000,  ..., 0.4627, 0.2314, 0.2431],
         [0.2588, 0.2667, 0.2078,  ..., 0.4510, 0.2235, 0.2471],
         [0.2392, 0.2549, 0.1686,  ..., 0.4039, 0.1961, 0.2314]],

        [[0.7176, 0.7804, 0.8078,  ..., 0.4627, 0.4549, 0.4275],
         [0.8706, 0.9412, 0.9686,  ..., 0.5176, 0.5098, 0.4824],
         [0.8353, 0.8863, 0.8784,  ..., 0.5294, 0.5176, 0

## Model

- Oroginal paper based on RESNET!

In [63]:
resnet152_model = resnet152(pretrained=False)
class HeadPoseModel_1(nn.Module):
    def __init__(self):
        super().__init__()
        self.resnet152_model = resnet152_model
        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(1000, 1000)
        self.act1 = nn.ReLU()
        self.linear2 = nn.Linear(1000, 500)
        self.act2 = nn.ReLU()
        self.linear3 = nn.Linear(500, 1)

    def forward(self, x):
      x = self.resnet152_model(x)
      x = self.flatten(x)
      x = self.act1(self.linear1(x))
      x = self.act2(self.linear2(x))
      x = self.linear3(x)
      return x

model = HeadPoseModel_1()
#print(model)
summary(model,(3,300,300))


e:\Programme\.conda\envs\CV_Head\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
e:\Programme\.conda\envs\CV_Head\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Layer (type:depth-idx)                   Output Shape              Param #
├─ResNet: 1-1                            [-1, 1000]                --
|    └─Conv2d: 2-1                       [-1, 64, 150, 150]        9,408
|    └─BatchNorm2d: 2-2                  [-1, 64, 150, 150]        128
|    └─ReLU: 2-3                         [-1, 64, 150, 150]        --
|    └─MaxPool2d: 2-4                    [-1, 64, 75, 75]          --
|    └─Sequential: 2-5                   [-1, 256, 75, 75]         --
|    |    └─Bottleneck: 3-1              [-1, 256, 75, 75]         75,008
|    |    └─Bottleneck: 3-2              [-1, 256, 75, 75]         70,400
|    |    └─Bottleneck: 3-3              [-1, 256, 75, 75]         70,400
|    └─Sequential: 2-6                   [-1, 512, 38, 38]         --
|    |    └─Bottleneck: 3-4              [-1, 512, 38, 38]         379,392
|    |    └─Bottleneck: 3-5              [-1, 512, 38, 38]         280,064
|    |    └─Bottleneck: 3-6              [-1, 512, 38, 38] 

Layer (type:depth-idx)                   Output Shape              Param #
├─ResNet: 1-1                            [-1, 1000]                --
|    └─Conv2d: 2-1                       [-1, 64, 150, 150]        9,408
|    └─BatchNorm2d: 2-2                  [-1, 64, 150, 150]        128
|    └─ReLU: 2-3                         [-1, 64, 150, 150]        --
|    └─MaxPool2d: 2-4                    [-1, 64, 75, 75]          --
|    └─Sequential: 2-5                   [-1, 256, 75, 75]         --
|    |    └─Bottleneck: 3-1              [-1, 256, 75, 75]         75,008
|    |    └─Bottleneck: 3-2              [-1, 256, 75, 75]         70,400
|    |    └─Bottleneck: 3-3              [-1, 256, 75, 75]         70,400
|    └─Sequential: 2-6                   [-1, 512, 38, 38]         --
|    |    └─Bottleneck: 3-4              [-1, 512, 38, 38]         379,392
|    |    └─Bottleneck: 3-5              [-1, 512, 38, 38]         280,064
|    |    └─Bottleneck: 3-6              [-1, 512, 38, 38] 

## Train Models

In [64]:
# set up DataLoader for training set
dataset = BIWI_Dataset()
loader = DataLoader(dataset, shuffle=True, batch_size=100)

In [65]:
loss_fn = nn.MSELoss()  # Mean squered error loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [66]:
n_epochs = 10

for epoch in range(n_epochs):
    for i, (X_batch, y_batch) in enumerate(loader):
        step = (epoch * len(loader)) + i + 1
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Save the model's state_dict every 1000 steps
        if step % 1000 == 0:
            save_mode_path = basepath + "/Models/model_1/model_checkpoint.pth"
            torch.save(model.state_dict(), save_mode_path)
    print(f'{epoch}, step: {step}, latest loss {loss}')

    print(f'Finished epoch {epoch}, latest loss {loss}')

save_mode_path = basepath + "/Models/model_1/model_checkpoint.pth"
torch.save(model.state_dict(), save_mode_path)


e:\Programme\.conda\envs\CV_Head\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
e:\Programme\.conda\envs\CV_Head\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([99])) that is different to the input size (torch.Size([99, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


0, step: 5, latest loss 0.6118510365486145
Finished epoch 0, latest loss 0.6118510365486145
1, step: 10, latest loss 0.42222583293914795
Finished epoch 1, latest loss 0.42222583293914795
2, step: 15, latest loss 0.2352713793516159
Finished epoch 2, latest loss 0.2352713793516159
3, step: 20, latest loss 0.25638386607170105
Finished epoch 3, latest loss 0.25638386607170105


### Load model

In [ ]:
# Initialize the model
model = TheModelClass(*args, **kwargs)
# Load the saved state_dict
model.load_state_dict(torch.load("path/to/save/model_checkpoint.pth"))
# Set the model to evaluation mode
model.eval()